In [84]:
import hashlib
import pandas as pd
import re
import logging
import openai
import json
import os
from typing import Optional, Dict, Any

logger = logging.getLogger(__name__)

In [101]:
def hash_md5(text: str) -> str:
    """
    Hash a text using md5
    
    Args:
        text()
    """
    return str(hashlib.md5(text.encode()).hexdigest())

def categorize_stay_duration(stay_duration: str) -> str:
    """
    Categorize stay duration into Short, Medium, Long
    
    Param
    """
    
    try:
        duration = int(re.sub(r"[^\d]", "", stay_duration))
        if duration < 2:
            return "Short"
        elif duration < 5:
            return "Medium"
        elif duration >= 5:
            return "Long"
    except Exception:
        return "Unknown"

def extract_addresses(description: str) -> Optional[Dict[str, str]]:
    """
    Extract structured address information using OpenAI's API.
    
    Args:
        description (str): The address description in English or Vietnamese.
        
    Returns:
        Optional[Dict[str, str]]: A dictionary with structured address components, or None if extraction fails.
    """

    prompt = f"""
    Analyze the following address description and extract the components into structured JSON. 
    Address descriptions may be in English or Vietnamese.

    Address: "{description}"

    Return a valid JSON object with the following keys:
    - "street": The street name or number, if available.
    - "ward": The ward name or number, if available.
    - "city": The city name.
    - "country": The country name.

    Ensure your response is a valid JSON object with no additional text or formatting, using null for missing fields. Please translate any English to corresponding meaningful Vietnamese text.
    
    For example:
    {{
      "street": "98 Đường Thông Thiên Học",
      "ward": "Phường 8",
      "city": "Đà Lạt",
      "country": "Việt Nam"
    }}
    """

        
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Your task is to extract structured JSON data based on the user input."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=200,
        )
        
        content = response.choices[0].message.content
        cleaned_content = content.strip("json").strip("").strip()

        # Parse the response content as JSON
        extracted_data = json.loads(cleaned_content)
        required_keys = {"street", "ward", "city", "country"}
        if required_keys.issubset(extracted_data.keys()):
            return extracted_data
        else:
            logger.error("Response JSON does not contain all expected keys.")
            return None

    except json.JSONDecodeError as e:
        logger.error(f"Failed to parse JSON from response: {e}")
        return None
    except Exception as e:
        logger.exception(f"An unexpected error occurred: {e}")
        return None

def extract_amenities(description: str) -> Optional[Dict[str, Any]]:
    """
    Extract structured amenities information using OpenAI's API.
    
    Args:
        description (str): The amenities description in English or Vietnamese.
        
    Returns:
        Optional[Dict[str, Any]]: A dictionary with structured amenities components, or None if extraction fails.
    """

    prompt = f"""
    Analyze the following amenities description and extract the components into structured JSON. 
    Amenities descriptions may be in English or Vietnamese.

    Amenities: "{description}"

    Return a valid JSON object with the following keys:
    - "has_air_conditioning": A boolean value indicating if WiFi is available.
    - "has_tv": A boolean value indicating if a TV is available.
    - "has_balcony": A boolean value indicating if a balcony is available.
    - "has_food_serving": A boolean value indicating if food is served at breakfast, lunch, or dinner.
    - "has_parking": A boolean value indicating if parking is available.
    - "has_hire_vehicle": A boolean value indicating if vehicle hire is available.

    Ensure your response is a valid JSON object with no additional text or formatting, using null for missing fields. Please translate any English to corresponding meaningful Vietnamese text.
    
    For example:
    {{
        "has_air_conditioning": true,
        "has_tv": true,
        "has_balcony": true,
        "has_food_serving": true,
        "has_parking": true,
        "has_hire_vehicle": true,
    }}
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Your task is to extract structured JSON data based on the user input."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=300,
        )
        
        content = response.choices[0].message.content
        cleaned_content = content.strip("json").strip("").strip()

        # Parse the response content as JSON
        extracted_data = json.loads(cleaned_content)
        required_keys = {
            "has_air_conditioning",
            "has_tv", "has_balcony",
            "has_food_serving",
            "has_parking",
            "has_hire_vehicle"
        }
        if required_keys.issubset(extracted_data.keys()):
            return extracted_data
        else:
            logger.error("Response JSON does not contain all expected keys.")
            return None

    except json.JSONDecodeError as e:
        logger.error(f"Failed to parse JSON from response: {e}")
        return None
    except Exception as e:
        logger.exception(f"An unexpected error occurred: {e}")
        return None

In [109]:
# Read reviews
reviews_file_path = "../scraper/booking/output/vn_hotels_reviews.parquet"
reviews = pd.read_parquet(reviews_file_path)

# Explode reviews column
reviews = reviews.explode("reviews")
reviews = pd.concat(
    [
        reviews.drop(["reviews"], axis=1), reviews["reviews"].apply(pd.Series)
    ],
    axis=1
)

# Rename columns
reviews = reviews.rename(
    columns={
        "hotel_name": "hotel_slug",
        "rating": "review_rating",
    }
)

# Concatenate into final review text
reviews["review_text_full"] = (
    reviews["review_title"].fillna("")
    + ". "
    + reviews["review_text_liked"].fillna("")
    + ". "
    + reviews["review_text_disliked"].fillna("")
)

# Add hotel_id, user_id
reviews["hotel_id"] = reviews.apply(lambda x: hash_md5(x["hotel_slug"]), axis=1)
reviews["user_id"] = reviews.apply(lambda x: hash_md5(str(x["username"])), axis=1)

# Reorder columns (move hotel_id, user_id to the front, remove hotel_slug, username)
reviews = reviews[
    [
        "hotel_id",
        "user_id",
        "review_rating",
        "review_title",
        "review_text_full",
        "stay_duration",
        "stay_type",
        "room_view",
        "username",
        "user_country"
    ]
]

# Categorize stay_duration (Short, Medium, Long)
reviews["stay_duration"] = reviews["stay_duration"].astype(str).apply(categorize_stay_duration)

# Convert to numeric
reviews["review_rating"] = pd.to_numeric(
    reviews["review_rating"], errors="coerce"
).astype(float)

# Reset index
reviews.reset_index(drop=True, inplace=True)
display(reviews.head())

,hotel_id,user_id,review_rating,review_title,review_text_full,stay_duration,stay_type,room_view,username,user_country
0,f8dd9fa66a227ffede2937d71ad05921,70bc55b31ea555f16b4b2cee1d5cf901,10.0,Exceptional,Exceptional. Hotel staff were friendly and hel...,Short,Couple,Deluxe Double Room,Heikki,Finland
1,f8dd9fa66a227ffede2937d71ad05921,527047c66c6af54086db833d12e1127d,10.0,Thank you Booking.Com. Good job,Thank you Booking.Com. Good job. Everything. W...,Medium,Family,Deluxe Queen Room with Two Queen Beds,Tractuyen,Vietnam
2,f8dd9fa66a227ffede2937d71ad05921,0b5a7b3db9977926290b4f518d3643a6,10.0,Exceptional,Exceptional. Good location and cheap with air ...,Short,Group,Deluxe Queen Room with Two Queen Beds,Neva,United Kingdom
3,f8dd9fa66a227ffede2937d71ad05921,97ae00172b8f4fab84922c3d73be1430,8.0,Very good,"Very good. It was a big room, very clean. We w...",Medium,Couple,Economy Quadruple Room - Basement,Jemma,United Kingdom
4,f8dd9fa66a227ffede2937d71ad05921,e35aa53ad2c0b1b2534562691e7b81bc,10.0,Great value for money and location,Great value for money and location. Staff were...,Short,Group,Deluxe Queen Room with Two Queen Beds,Marchessault,Slovakia


In [89]:
# Read hotels
hotels_file_path = "../scraper/booking/input/vn_hotels.csv"
hotels = pd.read_csv(hotels_file_path)
hotels = hotels[hotels["location"] == "Đà Lạt"]

# Extract structured address information
hotels["hotel_id"] = hotels.apply(lambda x: hash_md5(x["hotel_slug"]), axis=1)
hotels = hotels[
    [
        "hotel_id",
        "hotel_slug",
        "name_hotel",
        "descriptions",
        "address",
        "location",
        "country",
        "url_hotel",
    ]
]
hotels.reset_index(drop=True, inplace=True)
display(hotels.head())

,hotel_id,hotel_slug,name_hotel,descriptions,address,location,country,url_hotel
0,f8dd9fa66a227ffede2937d71ad05921,dalat-wind,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",Đà Lạt,vn,https://www.booking.com/hotel/vn/dalat-wind.vi...
1,fb401270b8babc2109a6dea17d1ebba2,tropicana-2,Miền Nhiệt Đới 2 Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Quảng trườn...","64 Phan Nhu Thach, ward 1, Đà Lạt, Việt Nam",Đà Lạt,vn,https://www.booking.com/hotel/vn/tropicana-2.v...
2,03adc0096493aa76d4e51842f8a15aa8,reddoorz-near-thung-lung-tinh-yeu,Raon Dalat,"Tọa lạc tại thành phố Đà Lạt, cách CLB chơi go...","46 Tran Khanh Du, Ward 8, Đà Lạt, Việt Nam",Đà Lạt,vn,https://www.booking.com/hotel/vn/reddoorz-near...
3,9bb7114f0c3873e5398496dd7e140f5e,royal-palace-thanh-pho-da-lat,Royal Palace,"Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...","Đường Hà Huy Tập 55 49A Đường Hà Huy Tập, Đà L...",Đà Lạt,vn,https://www.booking.com/hotel/vn/royal-palace-...
4,4a23f6c840d5b9f8d9f38fe289418a31,jolie-house-ap-da-thien,Jolie House,Tọa lạc ở thành phố Đà Lạt thuộc tỉnh Lâm Đồng...,"104 Phù Đổng Thiên Vương, Phường 8, Đà Lạt, Vi...",Đà Lạt,vn,https://www.booking.com/hotel/vn/jolie-house-a...


In [104]:
for idx, hotel in hotels.iterrows():
    # Extract structured address information
    print(hotel["address"])
    addresses_result = extract_addresses(hotel["address"])
    print(addresses_result)
    
    # Extract structured amenities information
    print(hotel["descriptions"])
    amenities_result = extract_amenities(hotel["descriptions"])
    print(amenities_result)
    break

Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Việt Nam
{'street': 'Lot R2 03-04, Golf Valley', 'ward': 'Phường 2', 'city': 'Đà Lạt', 'country': 'Việt Nam'}
Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hương 500 m, Dalat Wind Deluxe Hotel là khách sạn 2 sao có lễ tân 24 giờ, sảnh khách chung, Wi-Fi và chỗ đỗ xe riêng miễn phí. Tại khách sạn, tất cả các phòng đều có bàn làm việc, TV màn hình phẳng, ấm đun nước và phòng tắm riêng với chậu rửa vệ sinh (bidet). Một số phòng còn có ban công. Khách sạn phục vụ bữa sáng gọi món hàng ngày. Du khách có thể dùng bữa tại nhà hàng trong khuôn viên, nơi chuyên phục vụ các món nướng/BBQ. Du khách cũng có thể thư giãn trên sân hiên tắm nắng. Các điểm tham quan nổi tiếng gần Dalat Wind Deluxe Hotel bao gồm Vườn hoa Đà Lạt, Quảng trường Lâm Viên và Công viên Yersin Đà Lạt. Sân bay gần nhất là sân bay Liên Khương, cách chỗ nghỉ 23 km.
{'has_air_conditioning': None, 'has_tv': True, 'has_balcony': True, 'has_food_serving': True, 'has_parking': True, 'has_hire

In [105]:
for idx, hotel in hotels.iterrows():
    # Extract structured address information
    addresses_result = extract_addresses(hotel["address"])
    hotels.loc[idx, "street"] = addresses_result.get("street", None)
    hotels.loc[idx, "ward"] = addresses_result.get("ward", None)
    hotels.loc[idx, "city"] = addresses_result.get("city", None)
    hotels.loc[idx, "country"] = addresses_result.get("country", None)
    
    # Extract structured amenities information
    amenities_result = extract_amenities(hotel["descriptions"])
    hotels.loc[idx, "has_air_conditioning"] = amenities_result.get("has_air_conditioning", None)
    hotels.loc[idx, "has_tv"] = amenities_result.get("has_tv", None)
    hotels.loc[idx, "has_balcony"] = amenities_result.get("has_balcony", None)
    hotels.loc[idx, "has_food_serving"] = amenities_result.get("has_food_serving", None)
    hotels.loc[idx, "has_parking"] = amenities_result.get("has_parking", None)
    hotels.loc[idx, "has_hire_vehicle"] = amenities_result.get("has_hire_vehicle", None)

In [107]:
hotels.head()

,hotel_id,hotel_slug,name_hotel,descriptions,address,location,country,url_hotel,street,ward,city,has_air_conditioning,has_tv,has_balcony,has_food_serving,has_parking,has_hire_vehicle
0,f8dd9fa66a227ffede2937d71ad05921,dalat-wind,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/dalat-wind.vi...,"Lot R2 03-04, Golf Valley",Phường 2,Đà Lạt,None,True,True,True,True,None
1,fb401270b8babc2109a6dea17d1ebba2,tropicana-2,Miền Nhiệt Đới 2 Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Quảng trườn...","64 Phan Nhu Thach, ward 1, Đà Lạt, Việt Nam",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/tropicana-2.v...,64 Phan Nhu Thach,Phường 1,Đà Lạt,None,True,None,None,True,None
2,03adc0096493aa76d4e51842f8a15aa8,reddoorz-near-thung-lung-tinh-yeu,Raon Dalat,"Tọa lạc tại thành phố Đà Lạt, cách CLB chơi go...","46 Tran Khanh Du, Ward 8, Đà Lạt, Việt Nam",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/reddoorz-near...,46 Tran Khanh Du,Phường 8,Đà Lạt,None,None,None,None,True,True
3,9bb7114f0c3873e5398496dd7e140f5e,royal-palace-thanh-pho-da-lat,Royal Palace,"Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...","Đường Hà Huy Tập 55 49A Đường Hà Huy Tập, Đà L...",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/royal-palace-...,Đường Hà Huy Tập 55 49A,None,Đà Lạt,None,True,None,True,True,None
4,4a23f6c840d5b9f8d9f38fe289418a31,jolie-house-ap-da-thien,Jolie House,Tọa lạc ở thành phố Đà Lạt thuộc tỉnh Lâm Đồng...,"104 Phù Đổng Thiên Vương, Phường 8, Đà Lạt, Vi...",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/jolie-house-a...,104 Phù Đổng Thiên Vương,Phường 8,Đà Lạt,None,True,None,None,True,True


In [110]:
# Extract users
users = reviews[["username", "user_country"]].drop_duplicates()
users["user_id"] = users.apply(lambda x: hash_md5(str(x["username"])), axis=1)
users = users[["user_id", "username", "user_country"]]

# Reset index
users.reset_index(drop=True, inplace=True)
display(users.head())

,user_id,username,user_country
0,70bc55b31ea555f16b4b2cee1d5cf901,Heikki,Finland
1,527047c66c6af54086db833d12e1127d,Tractuyen,Vietnam
2,0b5a7b3db9977926290b4f518d3643a6,Neva,United Kingdom
3,97ae00172b8f4fab84922c3d73be1430,Jemma,United Kingdom
4,e35aa53ad2c0b1b2534562691e7b81bc,Marchessault,Slovakia


In [114]:
hotels.to_parquet("../data/vn_hotels.parquet", index=False)
reviews.to_parquet("../data/vn_hotels_reviews.parquet", index=False)
users.to_parquet("../data/vn_hotels_users.parquet", index=False)

In [115]:
pd.read_parquet("../data/vn_hotels.parquet").head()

,hotel_id,hotel_slug,name_hotel,descriptions,address,location,country,url_hotel,street,ward,city,has_air_conditioning,has_tv,has_balcony,has_food_serving,has_parking,has_hire_vehicle
0,f8dd9fa66a227ffede2937d71ad05921,dalat-wind,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/dalat-wind.vi...,"Lot R2 03-04, Golf Valley",Phường 2,Đà Lạt,None,True,True,True,True,None
1,fb401270b8babc2109a6dea17d1ebba2,tropicana-2,Miền Nhiệt Đới 2 Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Quảng trườn...","64 Phan Nhu Thach, ward 1, Đà Lạt, Việt Nam",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/tropicana-2.v...,64 Phan Nhu Thach,Phường 1,Đà Lạt,None,True,None,None,True,None
2,03adc0096493aa76d4e51842f8a15aa8,reddoorz-near-thung-lung-tinh-yeu,Raon Dalat,"Tọa lạc tại thành phố Đà Lạt, cách CLB chơi go...","46 Tran Khanh Du, Ward 8, Đà Lạt, Việt Nam",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/reddoorz-near...,46 Tran Khanh Du,Phường 8,Đà Lạt,None,None,None,None,True,True
3,9bb7114f0c3873e5398496dd7e140f5e,royal-palace-thanh-pho-da-lat,Royal Palace,"Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...","Đường Hà Huy Tập 55 49A Đường Hà Huy Tập, Đà L...",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/royal-palace-...,Đường Hà Huy Tập 55 49A,None,Đà Lạt,None,True,None,True,True,None
4,4a23f6c840d5b9f8d9f38fe289418a31,jolie-house-ap-da-thien,Jolie House,Tọa lạc ở thành phố Đà Lạt thuộc tỉnh Lâm Đồng...,"104 Phù Đổng Thiên Vương, Phường 8, Đà Lạt, Vi...",Đà Lạt,Việt Nam,https://www.booking.com/hotel/vn/jolie-house-a...,104 Phù Đổng Thiên Vương,Phường 8,Đà Lạt,None,True,None,None,True,True


In [3]:
import inspect
import pandas as pd

hotels = pd.read_parquet("../data/vn_hotels.parquet")
reviews = pd.read_parquet("../data/vn_hotels_reviews.parquet")
users = pd.read_parquet("../data/vn_hotels_users.parquet")

def print_schema(df):
    frame = inspect.currentframe().f_back
    variable_name = [k for k, v in frame.f_locals.items() if v is df][0]
    cols = df.columns
    print(variable_name)
    for col in cols:
        print(f"+-- {col}: " + str(df[col].dtype))
        
print_schema(hotels)
print_schema(reviews)
print_schema(users)

hotels
+-- hotel_id: object
+-- hotel_slug: object
+-- name_hotel: object
+-- descriptions: object
+-- address: object
+-- location: object
+-- country: object
+-- url_hotel: object
+-- street: object
+-- ward: object
+-- city: object
+-- has_air_conditioning: object
+-- has_tv: object
+-- has_balcony: object
+-- has_food_serving: object
+-- has_parking: object
+-- has_hire_vehicle: object
reviews
+-- hotel_id: object
+-- user_id: object
+-- review_rating: float64
+-- review_title: object
+-- review_text_full: object
+-- stay_duration: object
+-- stay_type: object
+-- room_view: object
+-- username: object
+-- user_country: object
users
+-- user_id: object
+-- username: object
+-- user_country: object


In [4]:
reviews.stay_type.value_counts()

stay_type
Couple            2255
Family            1616
Group             1200
Solo traveller    1034
Name: count, dtype: int64

In [6]:
reviews.stay_duration.value_counts()

stay_duration
Medium     3826
Short      2053
Long        226
Unknown       3
Name: count, dtype: int64